In [1]:
import h5py
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import awkward as awk
import os
import argparse
from heputl import logging as heplog
import gc
import numpy as np

import src.reader as read
import src.generator as gene
import src.selection as sele
import src.string_constants as stco
import src.util as util

In [2]:
N = int(1e2)
feat_mc = ['jet_pt_lead','jet_eta_lead','jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid']
dsids = stco.ds_ids_sig['tautau']        

In [3]:
# read one chunk to get dtype
dsid = dsids[0]
samples = read.read_samples_for_dsid(dsid,N=N,filtered=True)
weights = util.compute_w_samples(samples, dsid)
samples['dsid'] = np.int32(dsid)
samples['wt'] = weights
samples = samples[feat_mc]

2024-06-10 20:35:25 [reader.py:read_samples_for_dsid] reading samples for dsid 512198 from /eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.512198.MGPy8EG_FxFx_Ztautau_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root


In [4]:
field_types = [s._primitive for s in samples.type.content.contents]

In [5]:
field_types

['float32',
 'float32',
 'float32',
 'float64',
 'float64',
 'float64',
 'int32',
 'float32',
 'int32']

In [6]:
import numpy as np
dtype = np.dtype(list(zip(samples.fields,field_types)))

In [7]:
outfile.close()

NameError: name 'outfile' is not defined

In [8]:
outfile = h5py.File('test3.h5', 'w')

In [9]:
outfile.create_dataset("feature_names",data=[f.encode("ascii", "ignore") for f in feat_mc],dtype='S10')

<HDF5 dataset "feature_names": shape (9,), type "|S10">

In [10]:
events = outfile.create_dataset("events", shape=(0,), maxshape=(None,), dtype=dtype)

In [11]:
arr = awk.to_numpy(samples) # 

In [12]:
n_batch = len(arr)

In [13]:
events.resize(len(events) + n_batch, axis=0)
events[-n_batch:] = arr

In [65]:
stored = np.asarray(outfile.get('events'))

In [66]:
stored

array([( 483322.5  ,  1.535242  ,  1.9861127 , 8.27455819e-01, 4.49066702e-03, 1.67810813e-01,  0, -7.6693860e-03, 512198),
       ( 235857.48 ,  0.61030924,  0.19391856, 8.67007256e-01, 4.94682323e-03, 1.24805592e-01,  0,  2.5655649e-04, 512198),
       (  57461.965, -0.10597106,  2.4264474 , 3.77439141e-01, 3.88574507e-03, 5.50803468e-02,  0, -0.0000000e+00, 512198),
       (  42946.773, -1.002219  ,  0.6662743 , 5.78052461e-01, 9.48273484e-03, 1.28476799e-01,  0, -8.4549934e-02, 512198),
       (  52920.152, -2.1599023 ,  1.8263744 , 5.50543737e-05, 9.89185810e-01, 1.07495589e-02,  5,  1.8320690e-01, 512198),
       ( 328711.53 , -1.7040517 ,  0.1992111 , 3.13512646e-02, 6.48143470e-01, 3.17299575e-01,  5,  7.5993217e-03, 512198),
       ( 588772.6  , -1.7727304 , -1.8933219 , 9.04745638e-01, 3.30231548e-03, 9.18665081e-02,  0, -3.2865198e-03, 512198),
       (  74488.69 , -1.2563554 ,  3.118481  , 8.04134834e-05, 9.99806345e-01, 1.13178306e-04,  5, -1.9061382e-01, 512198),
       (

In [20]:
ff.close()

In [14]:
ff = h5py.File('test3.h5', 'r')

In [22]:
cols = list(f.decode('ascii') for f in ff.get('feature_names'))

In [23]:
cols

['jet_pt_lea',
 'jet_eta_le',
 'jet_phi_le',
 'jet_GN2_pu',
 'jet_GN2_pb',
 'jet_GN2_pc',
 'jet_truthf',
 'wt',
 'dsid']

In [17]:
retrieved = np.array(ff.get('events'))

In [15]:
import pandas as pd
evts = pd.DataFrame(np.array(ff.get('events')))

In [16]:
evts

,jet_pt_lead,jet_eta_lead,jet_phi_lead,jet_GN2_pu_lead,jet_GN2_pb_lead,jet_GN2_pc_lead,jet_truthflavExtended_lead,wt,dsid
0,483322.500000,1.535242,1.986113,0.827456,0.004491,0.167811,0,-0.007669,512198
1,235857.484375,0.610309,0.193919,0.867007,0.004947,0.124806,0,0.000257,512198
2,57461.964844,-0.105971,2.426447,0.377439,0.003886,0.055080,0,-0.000000,512198
3,42946.773438,-1.002219,0.666274,0.578052,0.009483,0.128477,0,-0.084550,512198
4,52920.152344,-2.159902,1.826374,0.000055,0.989186,0.010750,5,0.183207,512198
...,...,...,...,...,...,...,...,...,...
95,359393.937500,1.625967,2.313049,0.422511,0.098778,0.478635,0,0.005148,512198
96,41568.671875,-0.954784,0.220979,0.816178,0.020947,0.154823,0,0.114108,512198
97,31963.208984,0.943521,-0.429948,0.008079,0.892053,0.095712,5,-0.243511,512198
98,69307.281250,-1.173290,2.571591,0.874739,0.004641,0.115090,0,0.147572,512198


In [30]:
samples.type.content

RecordType([NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})], ['jet_pt_lead', 'jet_eta_lead', 'jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid'])

In [31]:
samples.type

ArrayType(RecordType([NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})], ['jet_pt_lead', 'jet_eta_lead', 'jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid']), 100, None)

In [39]:
dir(samples.type.content.contents[0])

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_typestr',
 '_is_equal_to',
 '_parameters',
 '_primitive',
 '_repr_args',
 '_str',
 '_str_categorical_begin',
 '_str_categorical_end',
 '_str_parameters',
 '_str_parameters_exclude',
 'copy',
 'is_equal_to',
 'parameter',
 'parameters',
 'primitive',
 'show']

In [38]:
print(samples.type.content.contents)

[NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})]


In [65]:
outfile = h5py.File('test.h5', 'w')

In [66]:
samples.fields

['jet_pt_lead',
 'jet_eta_lead',
 'jet_phi_lead',
 'jet_GN2_pu_lead',
 'jet_GN2_pb_lead',
 'jet_GN2_pc_lead',
 'jet_truthflavExtended_lead',
 'wt',
 'dsid']

In [68]:
list(zip(samples.fields,dtypes))

[('jet_pt_lead', 'float32'),
 ('jet_eta_lead', 'float32'),
 ('jet_phi_lead', 'float32'),
 ('jet_GN2_pu_lead', 'float64'),
 ('jet_GN2_pb_lead', 'float64'),
 ('jet_GN2_pc_lead', 'float64'),
 ('jet_truthflavExtended_lead', 'int32'),
 ('wt', 'float32'),
 ('dsid', 'str')]

In [18]:
dir(ds)

['__array__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache_props',
 '_d',
 '_dcpl',
 '_dxpl',
 '_e',
 '_extent_type',
 '_fast_read_ok',
 '_fast_reader',
 '_filters',
 '_id',
 '_is_empty',
 '_lapl',
 '_lcpl',
 '_readonly',
 '_selector',
 'asstr',
 'astype',
 'attrs',
 'chunks',
 'compression',
 'compression_opts',
 'dims',
 'dtype',
 'external',
 'fields',
 'file',
 'fillvalue',
 'fletcher32',
 'flush',
 'id',
 'is_scale',
 'is_virtual',
 'iter_chunks',
 'len',
 'make_scale',
 'maxshape',
 'name',
 'nbytes',
 'ndim',
 'parent',
 'read_dire

In [24]:
os.path.basename(ds.file.filename)

'test.h5'

In [22]:
h5py.Dataset

h5py._hl.dataset.Dataset